# **Agentic RAG** with LangChain and OpenAI

In [1]:
# Install the necessary packages
!pip install langchain -qU
!pip install langchain-openai -qU
!pip install langchain-community -qU
!pip install langchain-chroma -qU

!pip install duckduckgo-search -qU
!pip install pyowm -qU

!pip install langchainhub -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 964.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49

In [2]:
import os
from google.colab import userdata

### Initialize OpenAI LLM

In [3]:
from langchain_openai import ChatOpenAI

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize the ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

### Initialize Embedding Model

In [4]:
from langchain_openai import OpenAIEmbeddings

# Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

### Initialize DuckDuckGo Search Tool

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo search tool
search_tool = DuckDuckGoSearchRun()

search_tool

DuckDuckGoSearchRun()

### Initialize OpenWeatherMap Tool

In [19]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain.agents import load_tools

os.environ["OPENWEATHERMAP_API_KEY"] = userdata.get('OPENWEATHERMAP_API_KEY')

# Initialize OpenWeatherMap tool
weather = OpenWeatherMapAPIWrapper()

weather_tool = load_tools(["openweathermap-api"], llm)[0]

weather_tool

OpenWeatherMapQueryRun(api_wrapper=OpenWeatherMapAPIWrapper(owm=<pyowm.owm.OWM - API key=************************e2c88ea1, subscription type=free, PyOWM version=(3, 3, 0)>, openweathermap_api_key=None))

### Initialize RAG Tool

##### Load PDF Document

In [6]:
!pip install pypdf -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 2.5 MB/s eta 0:00:00


In [10]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("/content/codeprolk.pdf")

docs = loader.load()

In [11]:
len(docs)

4

##### Split Documents into Chunks

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Split the documents into chunks
splits = text_splitter.split_documents(docs)

##### Create Vector Store and Retriever

In [13]:
from langchain_chroma import Chroma

# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

# Create a retriever from the vector store
retriever = vectorstore.as_retriever()

##### Create Retrieval-Augmented Generation (RAG) Chain

In [14]:
from langchain.chains import RetrievalQA

# Retrieval qa chain
rag_chain = RetrievalQA.from_chain_type(
                                    llm=llm,
                                    chain_type="stuff",
                                    retriever=retriever)

In [15]:
from langchain.agents import Tool

rag_tool = Tool("RAG tool",
              func = rag_chain.run,
              description = "Use it to lookup information from the Veector Store")

### Define Prompt Template

In [16]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [17]:
# Set prompt template
from langchain_core.prompts import PromptTemplate

template= '''
          Answer the following questions as best you can. You have access to the following tools:

          {tools}

          Use the following format:

          Question: the input question you must answer
          Thought: you should always think about what to do
          Action: the action to take, should be one of [{tool_names}]. Always look first in Vector Store
          Action Input: the input to the action
          Observation: the result of the action
          ... (this Thought/Action/Action Input/Observation can repeat 2 times)
          Thought: I now know the final answer
          Final Answer: the final answer to the original input question

          Begin!

          Question: {input}
          Thought:{agent_scratchpad}
          '''

prompt = PromptTemplate.from_template(template)

### Create Built-in Agent

In [20]:
from langchain.agents import create_react_agent

# List of tools for the agent
tools = [search_tool, weather_tool, rag_tool]

# Create the agent using the LLM and the prompt template
agent = create_react_agent(tools = tools, llm = llm, prompt=prompt)

In [21]:
from langchain.agents import AgentExecutor

# Initialize the agent executor with the created agent and tools
agent_executor = AgentExecutor(tools=tools, agent=agent, handle_parsing_errors=True, verbose=True)

### Invoke the agent executor with an example question

In [23]:
response = agent_executor.invoke({"input":"courses in codeprolk"})

print(response['output'])



> Entering new AgentExecutor chain...
I should use the RAG tool to look up information about courses in Codeprolk from the Vector Store.
Action: RAG tool
Action Input: "courses in Codeprolk"CodePRO LK offers a wide array of free courses presented in Sinhala, catering to various proficiency levels from beginners to intermediates. Some key courses include Python GUI – Tkinter. The platform aims to expand its offerings to cover more advanced topics like artificial intelligence, cybersecurity, and advanced data analytics in the future.Action: RAG tool
Action Input: "Codeprolk courses"CodePRO LK offers a wide array of free courses presented in Sinhala, catering to various proficiency levels from beginners to intermediates. Some key courses include Python GUI – Tkinter. The platform aims to expand its offerings to cover more advanced topics like artificial intelligence, cybersecurity, and advanced data analytics in the future. Additionally, CodePRO LK engages its community through webinars

In [22]:
response = agent_executor.invoke({"input": "What's the weather in Kandy?"})

print(response['output'])



> Entering new AgentExecutor chain...
I should use the open_weather_map tool to find out the weather in Kandy.
Action: open_weather_map
Action Input: KandyIn Kandy, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 3.44 m/s, direction: 245°
Humidity: 79%
Temperature: 
  - Current: 23.56°C
  - High: 23.56°C
  - Low: 23.56°C
  - Feels like: 24.03°C
Rain: {}
Heat index: None
Cloud cover: 100%I now know the final answer
Final Answer: The current weather in Kandy is overcast clouds with a temperature of 23.56°C and 79% humidity.

> Finished chain.
The current weather in Kandy is overcast clouds with a temperature of 23.56°C and 79% humidity.
